FROM DLT SITE WILL TRY OUT SOME THINGS - https://dlthub.com/docs/getting-started

In [1]:
!pip install -U dlt

  Obtaining dependency information for dlt from https://files.pythonhosted.org/packages/6f/78/939b5eb62075a5e293261265d5022ee387cbfa86802750446eb8d9f716a2/dlt-0.4.3-py3-none-any.whl.metadata
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for gitpython>=3.1.29 from https://files.pythonhosted.org/packages/45/c6/a637a7a11d4619957cb95ca195168759a4502991b1b91c13d3203ffc3748/GitPython-3.1.41-py3-none-any.whl.metadata
  Obtaining dependency information for giturlparse>=0.10.0 from https://files.pythonhosted.org/packages/dd/94/c6ff3388b8e3225a014e55aed957188639aa0966443e0408d38f0c9614a7/giturlparse-0.12.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for hexbytes>=0.2.2 from https://files.pythonhosted.org/packages/73/99/f6beab7c3fe2ad62f6763a5e92af476225d74408b2588fe5614a67c63ee4/hexbytes-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for humanize>=4.4.0 from https://files.pythonhosted.org/packages/aa/2b/2a

In [1]:
!pip install "dlt[duckdb]"

  Obtaining dependency information for duckdb<0.10.0,>=0.6.1 from https://files.pythonhosted.org/packages/a6/7a/e15904563fa63d7d3cc542a697377486ddea8ba4914810391812ffefc9b1/duckdb-0.9.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/10.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.3 MB 330.3 kB/s eta 0:00:32
   ---------------------------------------- 0.1/10.3 MB 409.6 kB/s eta 0:00:26
   ---------------------------------------- 0.1/10.3 MB 595.3 kB/s eta 0:00:18
    --------------------------------------- 0.2/10.3 MB 748.1 kB/s eta 0:00:14
    --------------------------------------- 0.2/10.3 MB 846.9 kB/s eta 0:00:12
   - -------------------------------------- 0.4/10.3 MB 1.2 MB/s eta 0:00:09
   - -------------------------------------- 0.4/10.3 MB 1.2 MB/s eta 0:00:09
   --- ------------------------------------ 0.8/10.3 MB 1.9 MB/s eta 0:00:06
   --- ------

In [2]:
import dlt
from dlt.sources.helpers import requests

# Create a dlt pipeline that will load
# chess player data to the DuckDB destination
pipeline = dlt.pipeline(
    pipeline_name="chess_pipeline", destination="duckdb", dataset_name="player_data"
)
# Grab some player data from Chess.com API
data = []
for player in ["magnuscarlsen", "rpragchess"]:
    response = requests.get(f"https://api.chess.com/pub/player/{player}")
    response.raise_for_status()
    data.append(response.json())
# Extract, normalize, and load the data
load_info = pipeline.run(data, table_name="player")

In [3]:
data

[{'avatar': 'https://images.chesscomfiles.com/uploads/v1/user/3889224.121e2094.200x200o.361c2f8a59c2.jpg',
  'player_id': 3889224,
  '@id': 'https://api.chess.com/pub/player/magnuscarlsen',
  'url': 'https://www.chess.com/member/MagnusCarlsen',
  'name': 'Magnus Carlsen',
  'username': 'magnuscarlsen',
  'title': 'GM',
  'followers': 183584,
  'country': 'https://api.chess.com/pub/country/NO',
  'location': 'Norway',
  'last_online': 1707676983,
  'joined': 1282856720,
  'status': 'premium',
  'is_streamer': False,
  'verified': False,
  'league': 'Champion'},
 {'avatar': 'https://images.chesscomfiles.com/uploads/v1/user/28692936.b7d437d6.200x200o.293777c7e9fd.jpeg',
  'player_id': 28692936,
  '@id': 'https://api.chess.com/pub/player/rpragchess',
  'url': 'https://www.chess.com/member/rpragchess',
  'name': 'Praggnanandhaa Rameshbabu',
  'username': 'rpragchess',
  'title': 'GM',
  'followers': 6649,
  'country': 'https://api.chess.com/pub/country/IN',
  'location': 'CHENNAI',
  'last_

In [7]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# this lets us query data without adding schema prefix to table names
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# list all tables
display(conn.sql("DESCRIBE"))

stats_table = conn.sql("SELECT * FROM player").df()

display(stats_table)

┌────────────────┬─────────────┬─────────────────────┬──────────────────────┬──────────────────────────────┬───────────┐
│    database    │   schema    │        name         │     column_names     │         column_types         │ temporary │
│    varchar     │   varchar   │       varchar       │      varchar[]       │          varchar[]           │  boolean  │
├────────────────┼─────────────┼─────────────────────┼──────────────────────┼──────────────────────────────┼───────────┤
│ chess_pipeline │ player_data │ _dlt_loads          │ [load_id, schema_n…  │ [VARCHAR, VARCHAR, BIGINT,…  │ false     │
│ chess_pipeline │ player_data │ _dlt_pipeline_state │ [version, engine_v…  │ [BIGINT, BIGINT, VARCHAR, …  │ false     │
│ chess_pipeline │ player_data │ _dlt_version        │ [version, engine_v…  │ [BIGINT, BIGINT, TIMESTAMP…  │ false     │
│ chess_pipeline │ player_data │ player              │ [avatar, player_id…  │ [VARCHAR, BIGINT, VARCHAR,…  │ false     │
└────────────────┴─────────────┴

,avatar,player_id,aid,url,name,username,title,followers,country,location,last_online,joined,status,is_streamer,verified,league,_dlt_load_id,_dlt_id
0,https://images.chesscomfiles.com/uploads/v1/us...,3889224,https://api.chess.com/pub/player/magnuscarlsen,https://www.chess.com/member/MagnusCarlsen,Magnus Carlsen,magnuscarlsen,GM,183584,https://api.chess.com/pub/country/NO,Norway,1707676983,1282856720,premium,False,False,Champion,1707714360.9280941,0MNDqawMUEKL1Q
1,https://images.chesscomfiles.com/uploads/v1/us...,28692936,https://api.chess.com/pub/player/rpragchess,https://www.chess.com/member/rpragchess,Praggnanandhaa Rameshbabu,rpragchess,GM,6649,https://api.chess.com/pub/country/IN,CHENNAI,1707658628,1466301035,premium,False,False,Crystal,1707714360.9280941,njQM8u2LpPGj7Q


### POKEMON EXAMPLE FROM GOOGLE COLAB

https://colab.research.google.com/drive/1NfSB1DpwbbHX9_t5vlalBTf13utwpMGx?usp=sharing#scrollTo=A3NRS0y38alk

In [10]:
# create pipeline
pipeline = dlt.pipeline(pipeline_name="pokemon_pipeline", destination="duckdb", dataset_name="pokemon_data")

In [11]:
# get data from the source 

from dlt.sources.helpers import requests

POKEMON_URL = "https://pokeapi.co/api/v2/pokemon/"

data = requests.get(POKEMON_URL).json()["results"]

data

[{'name': 'bulbasaur', 'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
 {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
 {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
 {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
 {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
 {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
 {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
 {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
 {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
 {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
 {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
 {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
 {'name': 'weedle', 'url': 'https://pokeapi.co/api/v2/pokemon/13/'},
 {'name': 'kakuna', 'url': 'https://pokeapi.co/api/v2/pokemon/14/'},
 {'name': '

In [13]:
%%capture
# Normalize and load the data onto the locally created duckdb database 'pokemon_pipeline.duckdb'
pipeline.run(data, table_name='pokemon')

In [20]:
import duckdb

# a database 'chess_pipeline.duckdb' was created in working directory so just connect to it
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# this lets us query data without adding schema prefix to table names
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# list all tables
display(conn.sql("DESCRIBE"))

stats_table = conn.sql("SELECT * FROM pokemon").df()
display(stats_table)

┌──────────────────┬──────────────┬─────────────────────┬──────────────────────┬───────────────────────────┬───────────┐
│     database     │    schema    │        name         │     column_names     │       column_types        │ temporary │
│     varchar      │   varchar    │       varchar       │      varchar[]       │         varchar[]         │  boolean  │
├──────────────────┼──────────────┼─────────────────────┼──────────────────────┼───────────────────────────┼───────────┤
│ pokemon_pipeline │ pokemon_data │ _dlt_loads          │ [load_id, schema_n…  │ [VARCHAR, VARCHAR, BIGI…  │ false     │
│ pokemon_pipeline │ pokemon_data │ _dlt_pipeline_state │ [version, engine_v…  │ [BIGINT, BIGINT, VARCHA…  │ false     │
│ pokemon_pipeline │ pokemon_data │ _dlt_version        │ [version, engine_v…  │ [BIGINT, BIGINT, TIMEST…  │ false     │
│ pokemon_pipeline │ pokemon_data │ pokemon             │ [name, url, _dlt_l…  │ [VARCHAR, VARCHAR, VARC…  │ false     │
└──────────────────┴────────────

,name,url,_dlt_load_id,_dlt_id
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,1707715084.0987809,e/3yQbN3+OJ2Mw
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,1707715084.0987809,QHgbZVMEeJLxGw
2,venusaur,https://pokeapi.co/api/v2/pokemon/3/,1707715084.0987809,WTGoi2Bt6WhgKA
3,charmander,https://pokeapi.co/api/v2/pokemon/4/,1707715084.0987809,BG43VLaxmVloQA
4,charmeleon,https://pokeapi.co/api/v2/pokemon/5/,1707715084.0987809,jYcAHFBvYXh/Mg
5,charizard,https://pokeapi.co/api/v2/pokemon/6/,1707715084.0987809,aLPRwP6v0g3uQA
6,squirtle,https://pokeapi.co/api/v2/pokemon/7/,1707715084.0987809,FiA7pxS9+YiYcg
7,wartortle,https://pokeapi.co/api/v2/pokemon/8/,1707715084.0987809,DVCrmtdMkrspdQ
8,blastoise,https://pokeapi.co/api/v2/pokemon/9/,1707715084.0987809,WwOiiqvWSbJ56g
9,caterpie,https://pokeapi.co/api/v2/pokemon/10/,1707715084.0987809,aevF+U7tnl81yg


In [21]:
!pip install streamlit

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/60/6c/120ccf0af6d432025b7b3a77bf7f56a470c5e7e1ab4c1674bf8c73ed6c11/streamlit-1.31.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/c5/e4/7fcceef127badbb0d644d730d992410e4f3799b295c9964a172f92a469c7/altair-5.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/fa/2a/7f3714cbc6356a0efec525ce7a0613d581072ed6eb53eb7b9754f33db807/blinker-1.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/a2/91/2d843adb9fbd911e0da45fbf6f18ca89d07a087c3daa23e955584f90ebf4/cachetools-5.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf<5,>=3.20 from https://files.pythonhosted.org/packages/c1/00/c3ae19cabb36cfabc94ff0b102aac21b471c9f91a1357f8aafffb9efe

dlt pipeline quick_start show

this command will run pipeline in streamlit, this command can be run from terminal with actiavate conda env
https://dlthub.com/docs/getting-started

### load data from api tutorial

https://dlthub.com/docs/tutorial/load-data-from-an-api

In [28]:
import dlt
from dlt.sources.helpers import requests

# Specify the URL of the API endpoint
url = "https://api.github.com/repos/dlt-hub/dlt/issues"
# Make a request and check if it was successful
response = requests.get(url)
response.raise_for_status()

pipeline = dlt.pipeline(
    pipeline_name="github_issues",
    destination="duckdb",
    dataset_name="github_data",
)
# The response contains a list of issues
load_info = pipeline.run(response.json(), table_name="issues")

print(load_info)

Pipeline github_issues load step completed in 0.33 seconds
1 load package(s) were loaded to destination duckdb and into dataset github_data
The duckdb destination used duckdb:///C:\Users\Luka\Desktop\DESK\Studying\DE-zoomcamp\dlt-workshop\github_issues.duckdb location to store data
Load package 1707723310.8746843 is LOADED and contains no failed jobs


if we run same script twices it will append data because mode is append

In [32]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
stats_table = conn.sql("SELECT * FROM issues").df()
display(stats_table)

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,...,assignee__following_url,assignee__gists_url,assignee__starred_url,assignee__subscriptions_url,assignee__organizations_url,assignee__repos_url,assignee__events_url,assignee__received_events_url,assignee__type,assignee__site_admin
0,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/960,2129291323,PR_kwDOGvRYu85mlzbr,960,Introduce `replicate` write disposition,...,None,None,None,None,None,None,None,None,None,NaN
1,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/issues/957,2129181916,I_kwDOGvRYu85-6Lzc,957,publish a stable dependence freeze for core li...,...,None,None,None,None,None,None,None,None,None,NaN
2,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/956,2128750727,PR_kwDOGvRYu85mkOTB,956,Clarify info about GoodData in modelling tools...,...,None,None,None,None,None,None,None,None,None,NaN
3,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/954,2126291188,PR_kwDOGvRYu85mb7da,954,Add git to filesystem source 301,...,None,None,None,None,None,None,None,None,None,NaN
4,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/952,2126134270,PR_kwDOGvRYu85mbZvc,952,855 create bigquery adapter for dlt resources,...,https://api.github.com/users/Pipboyguy/followi...,https://api.github.com/users/Pipboyguy/gists{/...,https://api.github.com/users/Pipboyguy/starred...,https://api.github.com/users/Pipboyguy/subscri...,https://api.github.com/users/Pipboyguy/orgs,https://api.github.com/users/Pipboyguy/repos,https://api.github.com/users/Pipboyguy/events{...,https://api.github.com/users/Pipboyguy/receive...,User,False
5,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/950,2125375604,PR_kwDOGvRYu85mYxul,950,"Tweaked output to not include ""Found schema"" a...",...,None,None,None,None,None,None,None,None,None,NaN
6,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/issues/947,2123408014,I_kwDOGvRYu85-kKKO,947,Core extensions to support database replication,...,https://api.github.com/users/jorritsandbrink/f...,https://api.github.com/users/jorritsandbrink/g...,https://api.github.com/users/jorritsandbrink/s...,https://api.github.com/users/jorritsandbrink/s...,https://api.github.com/users/jorritsandbrink/orgs,https://api.github.com/users/jorritsandbrink/r...,https://api.github.com/users/jorritsandbrink/e...,https://api.github.com/users/jorritsandbrink/r...,User,False
7,https://api.github.com/repos/dlt-hub/dlt/iss

to solve that problem, we can use another mode / replace mode

In [31]:
import dlt
from dlt.sources.helpers import requests

# Specify the URL of the API endpoint
url = "https://api.github.com/repos/dlt-hub/dlt/issues"
# Make a request and check if it was successful
response = requests.get(url)
response.raise_for_status()

pipeline = dlt.pipeline(
    pipeline_name='github_issues',
    destination='duckdb',
    dataset_name='github_data',
)
# The response contains a list of issues
load_info = pipeline.run(
    response.json(),
    table_name="issues",
    write_disposition="replace"  # <-- Add this line
)

print(load_info)

Pipeline github_issues load step completed in 0.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset github_data
The duckdb destination used duckdb:///C:\Users\Luka\Desktop\DESK\Studying\DE-zoomcamp\dlt-workshop\github_issues.duckdb location to store data
Load package 1707723503.0704467 is LOADED and contains no failed jobs
